<a href="https://colab.research.google.com/github/Pakapakk/cybernews-rss-reader/blob/main/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSS-Finder

In [ ]:
%%capture
!pip install html5lib requests beautifulsoup4 pytz pandas dateutil feedparser

In [ ]:
%%capture
!pip install feedparser

In [ ]:
# TODO: REMOVE HTML TAGS FROM THE SCRAPPED DATA // Done
# TODO: SAVE TO JSON
# TODO: CSV <-> JSON CONVERTER

In [ ]:
import urllib.parse
import html5lib
import feedparser
import requests
from bs4 import BeautifulSoup
from dateutil import parser
from datetime import datetime
import pytz

import pandas as pd

In [ ]:
def find_feeds(urls):
    custom_feeds = {
        'https://thehackernews.com/' : 'https://feeds.feedburner.com/TheHackersNews',
        'https://www.bleepingcomputer.com/' : 'https://www.bleepingcomputer.com/feed/',
        'https://www.securityweek.com/': 'https://www.securityweek.com/feed/',  #  xml exits but keyword is not in the headers
        'https://www.microsoft.com/en-us/security/blog/': 'https://www.microsoft.com/en-us/security/blog/feed/',  #  xml exits but keyword is not in the headers
        'https://www.databreachtoday.com/': 'https://www.databreachtoday.com/rss-feeds',  #  xml exits but its only the path, not full url
        'https://packetstormsecurity.com/files/': 'https://rss.packetstormsecurity.com/files/',  #
        'https://packetstormsecurity.com/files/tags/exploit/': 'https://rss.packetstormsecurity.com/files/tags/exploit/',  #
        # 'https://edition.cnn.com/business/tech': 'http://rss.cnn.com/rss/cnn_tech.rss',  #  xml exits but its not on the page source
    }

    feeds = {}

    for url in urls:
        if url in custom_feeds:
            feeds[url] = custom_feeds[url]
            continue

        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            html = response.text
            tree = html5lib.parse(html, namespaceHTMLElements=False)

            base = tree.findall('.//base')
            base_url = base[0].attrib['href'] if base and 'href' in base[0].attrib else url

            links = tree.findall(".//head/link[@rel='alternate'][@type='application/atom+xml']") + \
                    tree.findall(".//head/link[@rel='alternate'][@type='application/rss+xml']")

            for link in links:
                href = link.attrib.get('href', '').strip()
                if href:
                    feed_url = urllib.parse.urljoin(base_url, href)
                    feeds[url] = feed_url
                    break

            if url not in feeds:
                for suffix in [
                    'feed', 'feed/', 'rss', 'atom', 'feed.xml',
                    '/feed', '/feed/', '/rss', '/atom', '/feed.xml',
                    'index.atom', 'index.rss', 'index.xml', 'atom.xml', 'rss.xml',
                    '/index.atom', '/index.rss', '/index.xml', '/atom.xml', '/rss.xml',
                    '.rss', '/.rss', '?rss=1', '?feed=rss2',
                ]:
                    try:
                        potential_feed = urllib.parse.urljoin(base_url, suffix)
                        response = requests.get(potential_feed)
                        # if response.status_code == 200 and ('xml' in response.headers.get('Content-Type', '') or 'rss' in response.headers.get('Content-Type', '')):
                        if ('xml' in response.headers.get('Content-Type', '') or 'rss' in response.headers.get('Content-Type', '')):
                            feeds[url] = potential_feed
                            break
                    except Exception:
                        continue

        except Exception as e:
            print(f"An error occurred with URL {url}: {e}")
            feeds[url] = None

    return feeds

In [ ]:
url_list = [
    'https://www.bleepingcomputer.com/',
    'https://thehackernews.com/',
    'https://www.theregister.com//security',
    'https://www.darkreading.com/',
    'https://www.zdnet.com//topic/security',
    'https://www.securityweek.com/',
    'https://securityaffairs.com/',
    # 'https://packetstormsecurity.com/files/',
    # 'https://packetstormsecurity.com/files/tags/exploit/',
    # 'https://www.databreachtoday.com/',
    'https://cyberscoop.com/',
    'https://cyberscoop.com/news/threats/cybercrime/',
    'https://www.microsoft.com/en-us/security/blog/',
    ]

In [ ]:
len(url_list)

10

In [ ]:
find_feeds(url_list)

{'https://www.bleepingcomputer.com/': 'https://www.bleepingcomputer.com/feed/',
 'https://thehackernews.com/': 'https://feeds.feedburner.com/TheHackersNews',
 'https://www.theregister.com//security': 'https://www.theregister.com/headlines.atom',
 'https://www.darkreading.com/': 'https://www.darkreading.com/rss.xml',
 'https://www.zdnet.com//topic/security': 'https://www.zdnet.com/rss.xml',
 'https://www.securityweek.com/': 'https://www.securityweek.com/feed/',
 'https://securityaffairs.com/': 'https://securityaffairs.com/feed',
 'https://cyberscoop.com/': 'https://cyberscoop.com/feed/',
 'https://cyberscoop.com/news/threats/cybercrime/': 'https://cyberscoop.com/feed/',
 'https://www.microsoft.com/en-us/security/blog/': 'https://www.microsoft.com/en-us/security/blog/feed/'}

In [ ]:
rss_list = [
    'https://www.bleepingcomputer.com/feed/',
    'https://feeds.feedburner.com/TheHackersNews',
    'https://www.theregister.com/headlines.atom',
    'https://www.darkreading.com/rss.xml',
    'https://www.zdnet.com/rss.xml',
    'https://www.securityweek.com/feed/',
    'https://securityaffairs.com/feed/',
    'https://cyberscoop.com/feed/',
    'https://www.microsoft.com/en-us/security/blog/feed/',
    'https://1stsecuritynews.com/feed/',
    'https://adamlevin.com/feed/',
    'https://www.alstonprivacy.com/feed/',
    'https://www.amazingsupport.co.uk/feed/',
    'https://www.andrewhay.ca/blog/feed',
    'https://architectsecurity.org/feed/',
    'https://blog.avast.com/feed',
    'https://medium.com/feed/@avi-lumelsky',
    'https://aws.amazon.com/blogs/security/feed/',
    'https://badcyber.com/feed/',
    'https://www.benthamsgaze.org/feed/',
    'https://www.blackhillsinfosec.com/blog/feed/',
    'https://blog.cadre.net/rss.xml',
    'https://www.canadiansecuritymag.com/feed/',
    'https://johnmasserini.com/feed/',
    'https://blogs.cisco.com/security/feed',
    'https://cloudsecurityalliance.org/feed',
    'https://www.cyberdefensemagazine.com/feed/',
    'http://cyber.uk/feed/',
    'https://defensivesecurity.org/feed/',
    'https://www.flyingpenguin.com/?feed=rss2',
    'https://feeds.feedblitz.com/GDataSecurityBlog-EN&x=1',
    'https://gbhackers.com/feed/',
    'http://feeds.feedburner.com/GoogleOnlineSecurityBlog',
    'https://www.grahamcluley.com/feed/',
    'https://www.hackingarticles.in/feed/',
    'https://www.hacknos.com/feed/',
    'https://medium.com/feed/@hpatton',
    'https://blogs.infoblox.com/feed/',
    'https://informationsecuritybuzz.com/feed/',
    'https://medium.com/feed/@infosecsherpa',
    'https://jeffsoh.blogspot.com/feeds/posts/default?alt=rss',
    'https://zeltser.com/blog/feed/',
    'http://luciusonsecurity.blogspot.com/feeds/posts/default',
    'https://marcoramilli.com/feed/',
    'https://medium.com/feed/@martinconnarty',
    'http://360tek.blogspot.com/feeds/posts/default?alt=rss',
    'http://360tek.blogspot.com/feeds/posts/default',
    'https://medium.com/feed/@nairuzabulhul',
    'http://feeds.feedburner.com/NoticeBored',
    'https://www.pindrop.com/feed/',
    'https://blog.qualys.com/feed',
    'https://rethinksecurity.io/posts/index.xml',
    'https://robert.penz.name/feed/',
    'http://feeds2.feedburner.com/RogersInfosecBlog',
    'https://medium.com/feed/@sai.51192',
    'https://secdevil.com/feed/',
    'https://www.securion.io/feed',
    'https://securityboulevard.com/feed/',
    'https://www.sentinelone.com/feed/',
    'https://taosecurity.blogspot.com/feeds/posts/default?alt=rss',
    'https://www.tcdi.com/blog/feed/',
    'http://tech-wreckblog.blogspot.com/feeds/posts/default?alt=rss',
    'https://848.co/feed/',
    'https://the-infosec.com/feed/',
    'https://blog.trailofbits.com/feed/',
    'http://feeds.trendmicro.com/TrendMicroResearch',
    'https://www.tripwire.com/state-of-security/feed/',
    'https://www.troyhunt.com/rss/',
    'http://www.veracode.com/blog/feed/',
    'https://www.vircom.com/feed/',
    'https://www.wikidsystems.com/blog/feeds/rss/',
    'https://www.winmagic.com/blog/feed/',
    'https://www.wisporg.com/blog-posts?format=RSS',
    'https://blog.rootshell.be/feed/',
]

In [ ]:
len(rss_list)

74

# Feed Aggregator


In [ ]:
def clean_text(html_text):
    """Removes HTML tags and returns plain text."""
    return BeautifulSoup(html_text, 'html.parser').get_text() if html_text else ""

def get_field(entry, field, default=""):
    """Retrieves a field from an entry, returns a default if field is missing."""
    return getattr(entry, field, default) if hasattr(entry, field) else default

def format_date_to_custom(date_str, timezone="Etc/GMT-7"):
    """Formats the date to 'Tue, 05 Nov 2024 11:03:00 +0700' format in GMT+7."""
    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
        # Convert to GMT+7 timezone
        target_timezone = pytz.timezone(timezone)
        date_obj_target = date_obj.replace(tzinfo=pytz.UTC).astimezone(target_timezone)
        return date_obj_target.strftime("%a, %d %b %Y %H:%M:%S %z")
    except ValueError:
        return date_str

In [ ]:
def parse_feed(feed_list):
    entries_data = []

    # for site, feed_url in feed_list.items():
    #     print(f"Fetching feed for: {site}")

    for feed_url in feed_list:
        # print(f"Fetching feed for: {site}")

        feed = feedparser.parse(feed_url)
        if feed.bozo:
            print(f"Failed to parse feed from {feed_url}")
            continue

        feed_source = clean_text(get_field(feed.feed, 'title'))
        feed_link = get_field(feed.feed, 'link')
        feed_description = clean_text(get_field(feed.feed, 'description'))
        fetched_timestamp = pd.Timestamp.now().strftime("%d-%m-%Y %H:%M:%S")

        entries_count = len(feed.entries)
        # print(f"Entries Count for {site}: {entries_count}\n")

        for entry in feed.entries:
            published_date = str(get_field(entry, 'published'))
            formatted_date = format_date_to_custom(published_date) if isinstance(published_date, str) and published_date else ""

            categories = [
                clean_text(tag['term']) if isinstance(tag, dict) else clean_text(tag)
                for tag in getattr(entry, 'tags', [])
                if tag
            ]
            categories_list = ", ".join(categories)

            content_encoded = get_field(entry, 'content', [{}])[0].get('value', '') or get_field(entry, 'description')
            content = clean_text(content_encoded) if content_encoded else clean_text(get_field(entry, 'summary',))
            entry_link = get_field(entry, 'link')

            entry_data = {
                "Title": clean_text(get_field(entry, 'title')),
                "Content": content,
                "Source": feed_source,
                "Category": categories_list,
                "Entry Published": formatted_date,
                "Entry Link": entry_link,
                "Fetch Timestamp": fetched_timestamp,
            }
            entries_data.append(entry_data)

    df = pd.DataFrame(entries_data)
    print(f"\nTotal Entries: {len(entries_data)}\n")
    return df

In [ ]:
# entries_data = parse_feed(find_feeds(url_list))

Fetching feed for: https://www.bleepingcomputer.com/
Entries Count for https://www.bleepingcomputer.com/: 13

Fetching feed for: https://thehackernews.com/
Entries Count for https://thehackernews.com/: 50

Fetching feed for: https://www.theregister.com//security
Entries Count for https://www.theregister.com//security: 50

Fetching feed for: https://www.darkreading.com/
Entries Count for https://www.darkreading.com/: 50

Fetching feed for: https://www.zdnet.com//topic/security
Entries Count for https://www.zdnet.com//topic/security: 20

Fetching feed for: https://www.securityweek.com/
Entries Count for https://www.securityweek.com/: 11

Fetching feed for: https://securityaffairs.com/
Entries Count for https://securityaffairs.com/: 10

Fetching feed for: https://cyberscoop.com/
Entries Count for https://cyberscoop.com/: 10

Fetching feed for: https://cyberscoop.com/news/threats/cybercrime/
Entries Count for https://cyberscoop.com/news/threats/cybercrime/: 10

Fetching feed for: https://w

In [ ]:
entries_data = parse_feed(rss_list)

Failed to parse feed from https://www.cyberdefensemagazine.com/feed/
Failed to parse feed from https://www.hacknos.com/feed/
Failed to parse feed from https://rethinksecurity.io/posts/index.xml
Failed to parse feed from https://www.winmagic.com/blog/feed/

Total Entries: 2292



# Save

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

def save_to_csv(df, filepath='/content/drive/My Drive/news_data/'):
    """Saves the DataFrame to a CSV file, handling potential duplicates."""

    if df.empty:
        print("No new data to save. Skipping save operation.")
        return

    csv_filepath = os.path.join(filepath, 'feed_input_demo.csv')

    if os.path.exists(csv_filepath):
        existing_df = pd.read_csv(csv_filepath)
        combined_df = pd.concat([existing_df, df]).drop_duplicates(subset=['Title', 'Entry Link'], keep='first')

        # Clear the file and save the updated DataFrame
        with open(csv_filepath, 'w+', encoding='utf-8') as f:
            f.truncate(0)  # Clear the file contents
        print(f"Added Entries: {len(combined_df) - len(existing_df)}")
        combined_df.to_csv(csv_filepath, index=False, encoding='utf-8', header=True)
    else:
        # Save the new DataFrame if the file doesn't exist
        df.to_csv(csv_filepath, index=False, encoding='utf-8', header=True)
        print(f"File created and saved with {len(df)} entries.")


In [ ]:
save_to_csv(entries_data)

File created and saved with 2292 entries.


In [ ]:
# entries_data.to_csv('feed_input.csv', encoding='utf-8', index=False)

In [ ]:
# entries_data.to_json('feed_input.json', orient='records', indent=4)

# Analysis

In [ ]:
news_df = pd.read_csv(('/content/drive/My Drive/news_data/feed_input.csv'))

In [ ]:
# news_df['Source'] = pd.to_datetime(news_df['Source'], dayfirst=True)
news_df.groupby(news_df['Source']).size()

,0
Source,
BleepingComputer,27
CyberScoop,20
DataBreachToday.com RSS Syndication,20
Exploit Files ≈ Packet Storm,16
Files ≈ Packet Storm,25
Microsoft Security Blog,26
Security Affairs,24
SecurityWeek,22
The Hacker News,102


In [ ]:
news_df['Source'] = news_df['Source'].str.replace('Latest news', 'Zdnet')

In [ ]:
news_df.to_csv('/content/drive/My Drive/news_data/feed_input.csv', index=False)

In [ ]:
news_df.groupby(news_df['Source']).size()

,0
Source,
BleepingComputer,27
CyberScoop,20
DataBreachToday.com RSS Syndication,20
Exploit Files ≈ Packet Storm,16
Files ≈ Packet Storm,25
Microsoft Security Blog,26
Security Affairs,24
SecurityWeek,22
The Hacker News,102


In [ ]:
# prompt: from news_df, make new df that contains only Sources = The Hacker News or The Register or Zdnet or darkreading

filtered_df = news_df[news_df['Source'].isin(['The Hacker News', 'The Register', 'Zdnet', 'darkreading'])]

,Title,Content,Source,Category,Entry Published,Entry Link,Fetch Timestamp
12,APT-K-47 Uses Hajj-Themed Lures to Deliver Adv...,The threat actor known as Mysterious Elephant ...,The Hacker News,NaN,"Fri, 22 Nov 2024 22:29:00 +0530",https://thehackernews.com/2024/11/apt-k-47-use...,23-11-2024 08:22:22
13,Russian Hackers Deploy HATVIBE and CHERRYSPY M...,Threat actors with ties to Russia have been li...,The Hacker News,NaN,"Fri, 22 Nov 2024 22:29:00 +0530",https://thehackernews.com/2024/11/russian-hack...,23-11-2024 08:22:22
14,China-Linked TAG-112 Targets Tibetan Media wit...,A China-linked nation-state group called TAG-1...,The Hacker News,NaN,"Fri, 22 Nov 2024 22:17:00 +0530",https://thehackernews.com/2024/11/china-linked...,23-11-2024 08:22:22
15,Unlocking Google Workspace Security: Are You D...,Google Workspace has quickly become the produc...,The Hacker News,NaN,"Fri, 22 Nov 2024 17:00:00 +0530",https://thehackernews.com/2024/11/the-importan...,23-11-2024 08:22:22
16,"Microsoft, Meta, and DOJ Disrupt Global Cyberc...","Meta Platforms, Microsoft, and the U.S. Depart...",The Hacker News,NaN,"Fri, 22 Nov 2024 15:07:00 +0530",https://thehackernews.com/2024/11/microsoft-me...,23-11-2024 08:22:22
...,...,...,...,...,...,...,...
505,"Bye bye, Wi-Fi: This low-cost adapter lets you...",Running ethernet wiring can be messy and expen...,Zdnet,NaN,"Fri, 17 Jan 2025 21:56:00 GMT",https://www.zdnet.com/article/bye-bye-wi-fi-th...,21-01-2025 07:19:33
506,"No, AI won't revolutionize shopping - but this...","As big brands chase the latest AI trends, they...",Zdnet,NaN,"Fri, 17 Jan 2025 21:24:00 GMT",https://www.zdnet.com/article/no-ai-wont-revol...,21-01-2025 07:19:33
507,Upgrading to Wi-Fi 7? This new router gave me ...,The Asus RT-BE86U is loaded with security tool...,Zdnet,NaN,"Fri, 17 Jan 2025 21:10:27 GMT",https://www.zdnet.com/article/upgrading-to-wi-...,21-01-2025 07:19:33
508,How to watch Super Bowl 2025: Every streaming ...,ZDNET found the cheapest ways to watch the gam...,Zdnet,NaN,"Fri, 17 Jan 2025 21:06:00 GMT",https://www.zdnet.com/article/how-to-watch-the...,21-01-2025 07:19:33


In [ ]:
filtered_df.groupby(news_df['Source']).size()

,0
Source,
The Hacker News,102
The Register,107
Zdnet,62
darkreading,100


In [ ]:
filtered_df.to_csv('/content/drive/My Drive/news_data/filtered_feed_input.csv', index=False)